In [17]:
from lxml import html as lhtml                         # Непосредственно для парсинга.
import requests                                        # Для получения html кода сайта.
from multiprocessing.dummy import Pool as ThreadPool   # Для реализации параллельной обкачки.
from multiprocessing import Lock, Value                # Для логгирования.
from math import ceil                                  # Для округления вверх.
from tqdm.notebook import tqdm                         # Для визуализации прогресса итерации по генератору.
import time                                            # Для ожидания окончания работы в  get_page().
from multiprocessing import Lock                       # Для логгирования.
import functools                                       # Для декоратора.

import numpy as np  # Для дебага, чтобы получать произвольные url'ы.
import random       # Для дебага, чтобы получать произвольные url'ы.

import pandas as pd # Непосредственно для формирования *.csv файла.
pd.options.display.max_columns=100

In [18]:
with open('authors.txt', 'r') as f:
    authors_id = list(map(int, f.read().split('\n')[:-1]))

In [19]:
way = '//div[@class="rd-listing-product-item rd-listing-product-item_size- rd-listing-product-item_type-"]/a'

In [20]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    r_get = requests.get(url, **kwargs)
    n_attempts -= 1
    
    while n_attempts > 0 and not r_get.ok:
            time.sleep(t_sleep)
            r_get = requests.get(url, **kwargs)
            n_attempts -= 1
    
    if n_attempts >= 0:
        return r_get   

In [21]:
def exception_counter(func):
    
    mutex = Lock()
    
    @functools.wraps(func)
    def wrapper(*args, **argv):   
        
        if not hasattr(wrapper, 'ex_count'):
            setattr(wrapper, 'ex_count', 0)
            
        try:
            result = func(*args, **argv)
        
        except:
            wrapper.ex_count += 1
            return 
        
        return result
    
    return wrapper

In [22]:
@exception_counter
def get_url(author):
    
    url_for_download = []

    page_url = "https://www.respublica.ru/authors/" + str(author)
    html = get_page(page_url)
    if html == None:
        return
    tree = lhtml.fromstring(html.text)
    books_per_page = tree.xpath('//span[@class="rd-listing-count__current"]')
    if books_per_page:
        books_per_page = int(books_per_page[0].text.split()[-1])
    else:
        return
    
    all_page_nums = ceil(int(tree.xpath('//span[@class="rd-listing-count__total"]')[0].text) / books_per_page)

    for page_num in range(all_page_nums):
        
        page_url = "https://www.respublica.ru/authors/" + str(author)
        html = get_page(page_url, params={'page': page_num})
        if html == None:
            return        
        tree = lhtml.fromstring(html.text)
        all_book_on_page = tree.xpath(way)

        for book in all_book_on_page:
            url_for_download.append("https://www.respublica.ru"+book.attrib['href'])
    return url_for_download

In [63]:
@exception_counter
def get_url(author):
    
    url_for_download = []

    page_url = "https://www.respublica.ru/authors/" + str(author)
    html = get_page(page_url)
    if html == None:
        return
    tree = lhtml.fromstring(html.text)
    all_book_on_page = True
    page_num = 0
    
    while all_book_on_page:
        
        html = get_page(page_url, params={'page': page_num})
        if html == None:
            return        
        
        tree = lhtml.fromstring(html.text)
        all_book_on_page = tree.xpath(way)

        for book in all_book_on_page:
            url_for_download.append("https://www.respublica.ru"+book.attrib['href'])
            
        page_num += 1
        
    return url_for_download

In [ ]:
%%time

with ThreadPool(10) as pool:
    authors_book_urls = pool.map(get_url, authors_id)
pool.join()

url_history_for_download = [url for sub_list in authors_book_urls for url in sub_list]

print("%i" % get_url.ex_count, "pages failed to download.")

In [ ]:
len(url_history_for_download)

In [ ]:
mutex = Lock()
n_processed = Value('i', 0)

def get_site(url):    
    
    if url == None:
        return
    field = {}
    html = get_page(url)
    if html == None:
        return
    tree = lhtml.fromstring(html.text)
    
    field["ID"] = tree.xpath('//span[@itemprop="sku"]')
    field["ID"] = field["ID"][0].text if field["ID"] else None
    field["URL"] = url
    field["Название"] = tree.xpath('//h1[@class="rd-page-product__title"]//text()')
    field["Название"] = field["Название"][0] if field["Название"] else None
    field["Автор"] = tree.xpath('//a[@itemprop="brand"]//text()')
    field["Автор"] = field["Автор"][0] if field["Автор"] else None
    
    preview_path = tree.xpath('//a[@class="download-pdf"]')
    field["Превью"] = "https://www.respublica.ru" + preview_path[0].attrib['href'] if preview_path else None
    
    field["Изображение"] = tree.xpath('//meta[@property="og:image"]')
    field["Изображение"] = field["Изображение"][0].attrib['content'] if field["Изображение"] else None
    field["Описание"] = tree.xpath('//div[@class="rd-page-product__desc-body"]//text()')
    field["Описание"] = ' '.join(field["Описание"]) if field["Описание"] else None
    field["Цена"] = tree.xpath('//span[@class="num"]//text()')
    field["Цена"] = int(field["Цена"][0].replace(' ','')) if field["Цена"] else None
    
    field["Цена (старая)"] = tree.xpath('//div[@class="rd-page-product__price-old"]//span//text()')
    field["Цена (старая)"] = field["Цена (старая)"][0] if field["Цена (старая)"] else None
    
    field["В наличии"] = bool(tree.xpath('//a[@class="rd-page-product__buy rd-page-product__buy_status_available"]'))
    
    categories = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//span[@itemprop="name"]//text()')
    field["Категория"] = '; '.join([item[1:] for item in categories]) if categories else None
    
    rating_block = tree.xpath('//span[@itemprop="aggregateRating"]')
    
    if rating_block:
        
        field["Число отзывов"] = rating_block[0].xpath('//meta[@itemprop="reviewCount"]')       
        field["Число отзывов"] = int(field["Число отзывов"][0].attrib['content']) if field["Число отзывов"] else None
        field["Число оценок"] = rating_block[0].xpath('//meta[@itemprop="ratingCount"]')
        field["Число оценок"] = int(field["Число оценок"][0].attrib['content']) if field["Число оценок"] else None
        field["Оценка"] = rating_block[0].xpath('//meta[@itemprop="ratingValue"]')
        field["Оценка"] = float(field["Оценка"][0].attrib['content']) if field["Оценка"] else None
    
    table = tree.xpath('//p[@class="rd-page-product__desc-param"]')
    table = table[0] if table else None

    names  = table.xpath('.//span[@itemprop="name"]/text()')
    values = table.xpath('.//*[@itemprop="value"]/text()')
    field.update(zip(names, values))  

    return field

@exception_counter
def get_site_wrapper(uid):
    
    try:    
        result = get_site(uid)
    except ex:
        raise(ex)
    
    with mutex:
        
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
            
    return result

In [ ]:
%%time

with ThreadPool(100) as pool:
    all_sites = pool.map(get_site_wrapper, url_history_for_download)
pool.join()
print("\n%i" % get_site_wrapper.ex_count, "pages couldn't be load.")

In [ ]:
all_sites = [site for site in all_sites if site] # Фильтруем от Nane'ов - сайтов, которые не получилось скачать.

In [ ]:
sites = pd.DataFrame(all_sites)
sites.head(10)

In [ ]:
sites.shape

In [ ]:
sites.isnull().sum()

In [ ]:
sites.to_csv('d.harazyan_hw_3.csv')